In [18]:
import os
import threading

In [19]:
print(f'Python process running with process id: {os.getpid()}')
total_threads = threading.active_count()
thread_name = threading.current_thread().name
 
print(f'Python is currently running {total_threads} thread(s)')
print(f'The current thread is {thread_name}')

Python process running with process id: 6203
Python is currently running 7 thread(s)
The current thread is MainThread


In [3]:
def hello_from_thread():
    print(f'Hello from thread {threading.current_thread()}!')
 
 
hello_thread = threading.Thread(target=hello_from_thread)
hello_thread.start()
 
total_threads = threading.active_count()
thread_name = threading.current_thread().name
 
print(f'Python is currently running {total_threads} thread(s)')
print(f'The current thread is {thread_name}')
 
hello_thread.join()

Hello from thread <Thread(Thread-7 (hello_from_thread), started 123145426575360)>!Python is currently running 8 thread(s)
The current thread is MainThread



GIL es particular para cada proceso.

“The global interpreter lock, abbreviated GIL and pronounced gill, is a controversial topic in the Python community. Briefly, the GIL prevents one Python process from executing more than one Python bytecode instruction at any given time. This means that even if we have multiple threads on a machine with multiple cores, a Python process can have only one thread running Python code at a time. In a world where we have CPUs with multiple cores, this can pose a significant challenge for Python developers looking to take advantage of multithreading to improve the performance of their application.

  Note Multiprocessing can run multiple bytecode instructions concurrently because each Python process has its own GIL.”

Excerpt From
Python Concurrency with asyncio
Matthew Fowler
This material may be protected by copyright.

Fibonacci secuencial

In [4]:
import time
 
def print_fib(number: int) -> None:
    def fib(n: int) -> int:
        if n == 1:
            return 0
        elif n == 2:
            return 1
        else:
            return fib(n - 1) + fib(n - 2)
 
    print(f'fib({number}) is {fib(number)}')
 
 
def fibs_no_threading():
    print_fib(40)
    print_fib(41)
 
 
start = time.time()
 
fibs_no_threading()
 
end = time.time()
 
print(f'Completed in {end - start:.4f} seconds.')

fib(40) is 63245986
fib(41) is 102334155
Completed in 42.5858 seconds.


In [5]:
def fibs_with_threads():
    fortieth_thread = threading.Thread(target=print_fib, args=(40,))
    forty_first_thread = threading.Thread(target=print_fib, args=(41,))
 
    fortieth_thread.start()
    forty_first_thread.start()
 
    fortieth_thread.join()
    forty_first_thread.join()
 
 
start_threads = time.time()
 
fibs_with_threads()
 
end_threads = time.time()
 
print(f'Threads took {end_threads - start_threads:.4f} seconds.')

fib(40) is 63245986
fib(41) is 102334155
Threads took 42.9687 seconds.


“The global interpreter lock is released when I/O operations happen. This lets us employ threads to do concurrent work when it comes to I/O, but not for CPU-bound Python code itself (there are some notable exceptions that release the GIL for CPU-bound work in certain circumstances, and we’ll look at these in a later chapter).”

Excerpt From
Python Concurrency with asyncio
Matthew Fowler
This material may be protected by copyright.

In [7]:
import requests
 
 
def read_example() -> None:
    response = requests.get('https://www.example.com')
    print(response.status_code)
 
 
sync_start = time.time()
 
read_example()
read_example()
 
sync_end = time.time()
 
print(f'Running synchronously took {sync_end - sync_start:.4f} seconds.')

200
200
Running synchronously took 0.2982 seconds.


In [11]:
thread_1 = threading.Thread(target=read_example)
thread_2 = threading.Thread(target=read_example)
 
thread_start = time.time()
 
thread_1.start()
thread_2.start()
 
print('All threads running!')
 
thread_1.join()
thread_2.join()
 
thread_end = time.time()
 
print(f'Running with threads took {thread_end - thread_start:.4f} seconds.')

All threads running!
200
200
Running with threads took 0.1283 seconds.


[Python Concurrency Learning Paths](https://superfastpython.com/learning-paths/#Asyncio_Learning_Path)

Siguiendo los ejemplos de [*Understanding the Python GIL*](https://www.youtube.com/watch?v=Obt-vMVdM8s&t=567s&ab_channel=DavidBeazley)

In [2]:
import time

In [3]:
def countdown(n: int) -> None:
    while n > 0:        
        n -= 1
        
COUNT = 100_000_000
start = time.time()
countdown(COUNT)
end = time.time()
print(f'Completed in {end - start:.4f} seconds.')
        

Completed in 5.0609 seconds.


El primer intento que se puede hacer para mejorar el rendimiento de la función countdown es utilizar la función threading.Thread para ejecutar la función en un hilo separado. A continuación se muestra el código que muestra cómo se puede hacer esto:

In [5]:
from threading import Thread

In [9]:
t1 = Thread(target=countdown, args=(COUNT//2,))
t2 = Thread(target=countdown, args=(COUNT//2,))
start = time.time()
t1.start()
t2.start()
t1.join()
t2.join()
end = time.time()
print(f'Completed in {end - start:.4f} seconds.')

Completed in 5.3844 seconds.


Corremos de nuevo con un sólo procesador habilitado

In [7]:
t1 = Thread(target=countdown, args=(COUNT//2,))
t2 = Thread(target=countdown, args=(COUNT//2,))
start = time.time()
t1.start()
t2.start()
t1.join()
t2.join()
end = time.time()
print(f'Completed in {end - start:.4f} seconds.')

Completed in 11.6323 seconds.


Ahora vamos a correr el código lanzando además otro script de Python, `scripts/async/spin.py` de esta forma:

In [14]:
from pathlib import Path

SCRIPTS_PATH = Path.cwd().parent.parent / 'scripts'
TOPIC_PATH = 'async'

PROGRAM_PATH = SCRIPTS_PATH / TOPIC_PATH

In [15]:
from subprocess import Popen

p = Popen(['python', PROGRAM_PATH / 'spin.py'])

t1 = Thread(target=countdown, args=(COUNT//2,))
t2 = Thread(target=countdown, args=(COUNT//2,))
start = time.time()
t1.start()
t2.start()
t1.join()
t2.join()
end = time.time()
print(f'Completed in {end - start:.4f} seconds.')

p.terminate()

Completed in 3.1687 seconds.


Este comportamiento absolutamente anti intuitivo se debe al GIL (*Global Interpreter Lock*)

Qué es un lock?


In [ ]:
Un *lock* (o cerrojo) es una primitiva de sincronización utilizada en programación concurrente para controlar el acceso a un recurso compartido por múltiples hilos o procesos. Un *lock* asegura que solo un hilo o proceso pueda acceder al recurso en un momento dado, evitando condiciones de carrera (*race conditions*).

En Python, los *locks* son proporcionados por el módulo `threading` y se utilizan para proteger secciones críticas del código. Un ejemplo típico es cuando varios hilos intentan modificar una variable compartida al mismo tiempo. Al usar un *lock*, se puede garantizar que solo un hilo tenga acceso exclusivo a esa variable mientras realiza las modificaciones.
 

In [16]:
import sys

sys.getswitchinterval()

0.005